In [1]:
import numpy as np
import random 
np.random.seed(0)
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd

In [ ]:
class ResourceAllocationProblem():
    def __init__(self, space, resources, maxima, constraints):
        """
        """
        self.space = space
        self.resources = resources
        self.constraints = constraints
        self.states = self.find_states()
        self.constraints = constraints

    def find_states(self):
        return {}

    def generate_random_real_points( number_of_points, accumulate = False, negative_numbers = False ):
        x, y = [], []
        for i in range(number_of_points):
            x_incr, y_incr = random.random(), random.random(), 
            if len(x) == 0:
                x.append(x_incr)
                y.append(y_incr)
            else:
                x.append(x_incr+x[-1])
                if accumulate:
                    y.append(y_incr+y[-1])
                else:
                    y.append(y_incr)
        x = np.array(x)
        y = np.array(y)
        return  dict(zip(x, y)) 

    def generate_random_convex_function( number_of_points ):
        return integral_function(generate_random_real_points( number_of_points, accumulate = False ))

    def generate_several_rcf( number_of_functions, number_of_points ):
        return np.array([generate_random_convex_function( number_of_points ) for i in range(number_of_functions)])

    def piecewise_linear_function(points):
        # Sort the dictionary by x values (keys)
        sorted_points = sorted(points.items())
        def final_function( t ):
        # If t is outside the range of x values, return the corresponding endpoint y value
            if t <= sorted_points[0][0]:
                return sorted_points[0][1]
            if t >= sorted_points[-1][0]:
                return sorted_points[-1][1]
            
            # Find the two points that enclose t
            for i in range(len(sorted_points) - 1):
                x1, y1 = sorted_points[i]
                x2, y2 = sorted_points[i+1]
                
                if x1 <= t <= x2:
                    # Compute the y value using the equation of the line
                    return (y2 - y1) / (x2 - x1) * (t - x1) + y1
        return final_function


    def integral_function(points):
        # Sort the dictionary by x values (keys)
        sorted_points = sorted(points.items())
        
        def final_function( t ):
        # If t is outside the range of x values, return the accumulated value at the endpoints
            if t <= sorted_points[0][0]:
                return 0  # Assuming starting value is 0
            if t >= sorted_points[-1][0]:
                accumulated_value = 0
                for i in range(len(sorted_points) - 1):
                    accumulated_value += sorted_points[i][1] * (sorted_points[i+1][0] - sorted_points[i][0])
                return accumulated_value
            
            # Compute the accumulated value of the function at t
            accumulated_value = 0
            for i in range(len(sorted_points) - 1):
                x1, y1 = sorted_points[i]
                x2, y2 = sorted_points[i+1]
                
                if x1 <= t <= x2:
                    accumulated_value += y1 * (t - x1)
                    return accumulated_value
                else:
                    accumulated_value += y1 * (x2 - x1)
        return final_function

    def plot_function( real_function ):

        x_values = np.linspace(-10, 10, 400)
        y_values = np.array([real_function(t) for t in x_values])

        # Plot the function
        plt.figure(figsize=(8, 6))
        plt.plot(x_values, y_values, label="f(x) = real function", color='blue')
        plt.title("Plot of real_function")
        plt.xlabel("x")
        plt.ylabel("f(x)")
        plt.grid(True)
        plt.legend()
        plt.show()



    def evaluate_functions_at_points( functions, points ):
        try:
            ret = [functions[j](points[j]) for j in range(len(functions))]

            return ret
        except:
            try:
                # print(functions[0])
                # print(points[0][0])
                # print(functions[0](points[0][0]))
                ret = [functions[j](points[0][j]) for j in range(len(functions))]
                return ret
            except:
                print(f'counldnt evaluate fuctions {functions} at points {points}')

    # Define the neural network
    class Net(nn.Module):
        def __init__(self, num_functions):
            super(Net, self).__init__()
            self.fc1 = nn.Linear(num_functions, 128)
            self.fc2 = nn.Linear(128, 128)
            # self.fc3 = nn.Linear(64, 128)
            self.fc4 = nn.Linear(128, num_functions)
        
        def forward(self, x):
            x = torch.relu(self.fc1(x))
            x = torch.relu(self.fc2(x))  
            # x = torch.relu(self.fc3(x))  
            x = torch.relu(self.fc4(x))  
            return x


    # Loss function with penalty for constraint violation
    def custom_loss(output, input, functions, N, print_bool):
        # evaluation = list(np.array(evaluate_functions_at_points( functions, output )) - np.array(evaluate_functions_at_points( functions, input )) )
        
        # evaluation = [x - y for x, y in evaluate_functions_at_points( functions, output ), evaluate_functions_at_points( functions, input )]
        evaluation = evaluate_functions_at_points( functions, output )
        for i in range(len(evaluation)):
            evaluation[i] = evaluation[i] - evaluate_functions_at_points( functions, input )[i]
        loss = 100 - sum(evaluation)
        if output.sum() > N:
            penalty = (output.sum() - N)**2
        else:
            penalty = (output.sum() - N)**2 - (output.sum() - N)**2
        if print_bool:
            print('evaluation', evaluation)
            print('loss', loss)
            print('penalty', penalty)
        return loss + penalty

    def initialize_setting( number_of_functions, number_of_points, max_resource ):
        functions = generate_several_rcf( number_of_functions, number_of_points )
        N = max_resource
        return functions, N

    functions, N = initialize_setting( 10, 50, 200 )


    net = Net(len(functions))
    optimizer = optim.Adam(net.parameters(), lr=0.0008)
    # scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)

    # Dummy input (can be your actual data or a tensor of ones for simplicity)
    input_data = torch.ones(1, len(functions))
    losses = [0, 1, 2]
    constraint_violations = []

        # Training loop
    def train( num_epochs, print_bool = False, plot_bool = False ):
        for epoch in range(num_epochs):
            optimizer.zero_grad()
            output = net(input_data)
            # loss = custom_loss(output, input_data, functions, N)
            loss = custom_loss(output, input_data, functions, N, print_bool=False)
            loss.backward()
            optimizer.step()
            # scheduler.step()    
            # Log metrics
            if losses[-1] == losses[-2] and losses[-1] == losses[-3]:
                print('yea')
                break
            losses.append(loss.item())
            if output.sum() > N:
                constraint_violations.append(abs(output.sum().item() - N))
            else:
                constraint_violations.append(0)

        if plot_bool:
            plt.figure(figsize=(12, 5))
            plt.subplot(1, 2, 1)
            plt.plot(losses)
            plt.title("Loss over training")
            plt.xlabel("Epochs")
            plt.ylabel("Loss")
            plt.subplot(1, 2, 2)
            plt.plot(constraint_violations)
            plt.title("Constraint Violation over training")
            plt.xlabel("Epochs")
            plt.ylabel("Absolute difference from N")
            plt.tight_layout()
            plt.show()
        if print_bool:
            result = net(input_data)
            print(result)
            print(sum(sum(result)))
            print(evaluate_functions_at_points(functions, result))


def Agent():
    def __init__(self, model, strategy):
        """
        """
        self.model = model
        self.strategy = strategy


class State:
    def __init__(self):
        """
        
        """
        self.data = np.zeros()
        self.hash_val = None
        self.end = None

    # compute the hash value for one state, it's unique
    def hash(self):
        if self.hash_val is None:
            print(self.hash_val)
        return self.hash_val

    def is_end(self):
        #TODO add end condition(?)
        self.end = False
        return self.end
    
    def next_state(self, ):
        new_state = State()
        return new_state

    # print the board
    def print_state(self):
        pass



In [ ]:
class Plot():
    def __init__(self, current_crop, ):
        self.current_crop = current_crop

        
class Resource():
    def __init__(self, type, ):
        self.type = type

class Crop():
    def __init__(self, productivity, growth_time, resistance):
        pass

class Category():
    def __init__(self):
        pass

class Agent():
    def __init__(self, money):
        self.money = money

# class Money():
#     def __init__(self):
#         pass
# class Nutrient():
#     def __init__(self):
#         pass
# class Defensive():
#     def __init__(self):
#         pass
# class Optimizer():
#     def __init__(self):
#         pass


In [ ]:
# TRASH




# def generate_random_nondecreasing_function( number_of_points,  ):
#     x, y = [], []
#     for i in range(number_of_points):
#         x_incr, y_incr = random.random(), random.random(), 
#         if len(x) == 0:
#             x.append(x_incr)
#             y.append(y_incr)
#         else:
#             x.append(x_incr+x[-1])
#             y.append(y_incr+y[-1])
#     x = np.array(x)
#     y = np.array(y)
#     print(x)
#     print(y)
#     def final_funcion( x, y, t ):
#         assert x[0] <= t
#         assert x[-1] >= t
        
#     return final_funcion
